In [1]:
import numpy as NP

#########################################################################
# Utilities

def oddFactorial(k):
    f = k
    while k >= 3:
        k -= 2
        f *= k
    return f

def oddOnEvenFactorial(k):

  f = 1
  for m in range(1,k+1):
    f *= (2*m+1)/(2.0*m)
  return f

#########################################################################
# Basic legendre polynomial computations
# It is not recommended to use them in a loop in l, l/m

def legendre(l,x,normalized=True,getDerivative=False):

  if NP.amin(x) < -1.0 or NP.amax(x) > 1.0:
    raise ValueError('abscissa <-1 or >1')
 
  if normalized:
    if hasattr(x,'shape'):
      p_l = 1/NP.sqrt(2.0)*NP.ones(x.shape)
    else:
      p_l = 1/NP.sqrt(2.0)*NP.ones((1,))
  else:
    if hasattr(x,'shape'):
      p_l = 1.0*NP.ones(x.shape)
    else:
      p_l = 1.0*NP.ones((1,))
  if l == 0:
    if getDerivative:
      return p_l,NP.zeros(p_l.shape)
    else:
      return p_l

  # Term l=1
  if normalized:
    p_ll = x*NP.sqrt(1.5)
  else:
    p_ll = p_l*x

  if getDerivative:
    if normalized:
      dp_ll = NP.sqrt(1.5)*NP.ones(p_ll.shape)
    else:
      dp_ll = NP.ones(p_ll.shape)

  if l == 1:
    if getDerivative:
      return p_ll,dp_ll
    else:
      return p_ll

  # Recursion
  for L in range(2,l+1):
    result = iterLegendre(L,x,p_ll,p_l,normalized)
    if getDerivative:
      resD = iterLegendreDerivative(L,x,result,p_ll,normalized)    
    p_l  = p_ll
    p_ll = result

  if getDerivative:
    return result,resD
  else:
    return result

def associatedLegendre(l,m,x,normalized=True):

  ''' Returns the associated Legendre polynomial,
      normalized by sqrt((2l+1)/2)*sqrt((l-m)!/(l+m)!) if required

      Negative ms calls the function with positive m.

      Recursion is performed from Pmm towards Plm (as |m|<=l).
  '''

  if NP.amin(x) < -1.0 or NP.amax(x) > 1.0:
    raise ValueError('abscissa <-1 or >1')
  if abs(m)>l:
    raise ValueError('degree m not compatible with order l : m=%d, l=%d'%(m,l))
 
  # Negative ms
  if (m<0):
    if normalized:
      return (-1)**abs(m)*associatedLegendre(l,abs(m),x,normalized)
    else:
      return (-1)**abs(m)/NP.product(NP.arange(l-abs(m)+1.e0,l+abs(m)+1.e0))*associatedLegendre(l,abs(m),x,normalized)

  # Initial term of the recursion
  if m == 0:
    if normalized:
      if hasattr(x,'shape'):
        p_mm = 1/NP.sqrt(2.0)*NP.ones(x.shape)
      else:
        p_mm = 1/NP.sqrt(2.0)*NP.ones((1,))
    else:
      if hasattr(x,'shape'):
        p_mm = 1.e0*NP.ones(x.shape)
      else:
        p_mm = 1.e0*NP.ones((1,))
  else:
    s = 1
    if m & 1:
      s = -1
    z = NP.sqrt(1-x**2)
    if normalized:
      p_mm = s*z**m*NP.sqrt(oddOnEvenFactorial(m)/2.0)
    else:
      p_mm = s*oddFactorial(2*m-1)*z**m

  if m == l:
    return p_mm

  # Next terms
  p_lm  = p_mm
  if normalized:
    p_llm = p_mm*x*NP.sqrt(2.0*m+3)
  else:
    p_llm = p_mm*x*(2*m+1)

  if l == m+1:
    return p_llm

  # Recursion
  for L in range(m+2,l+1):
    result = iterLAssociatedLegendre(L,m,x,p_llm,p_lm,normalized)
    p_lm   = p_llm
    p_llm  = result

  return result

def sphericalHarmonic(ls,ms,theta,phi,normalized=True):

  ''' theta and phi can be given as 1D vectors or meshgrid arrays '''
  th = theta
  f  = phi

  if not hasattr(ls,'__len__'):
    ls = NP.array([ls])
  if not hasattr(ms,'__len__'):
    ms = NP.array([ms])
  if hasattr(th,'__len__'):
    if th.ndim > 1:
      th = th[:,0]
  else:
    th = NP.array([th])
  if hasattr(f,'__len__'):
    if f.ndim > 1:
      f = f[0,:]
  else:
    f = NP.array([f])

  # Plm = associatedLegendre(l,m,NP.cos(th),normalized)
  Plm = associatedLegendre_grid(ls,ms,theta,normalized)
  Eim = NP.exp(1j*ms[:,NP.newaxis]*f[NP.newaxis,:])
  if normalized:
    Eim /= NP.sqrt(2.e0*NP.pi)

  return Plm[...,NP.newaxis]*Eim[NP.newaxis,:,NP.newaxis,:]




#########################################################################
# Iterations method to go from Pl to Pl+1, Plm to Pl+1m

def iterLegendre(L,x,PLm1,PLm2,normalized=True):
  ''' given P_(L-1) and P_(L-2), returns P_(L)'''

  l = float(L)
  if normalized:
    result = (2*l-1)/l*NP.sqrt((2*l+1)/(2*l-1))*x*PLm1 - (l-1)/l*NP.sqrt((2*l+1)/(2*l-3.0))*PLm2
  else:
    result = (x*(2*l-1)*PLm1 - (l-1)*PLm2)/l
  return result

def iterLegendreDerivative(L,x,PL,PLm1,normalized=True):
  ''' given P_(L-1) and P_(L), returns P'_(L)'''

  l = float(L)
  if normalized:
    result        = -l*(x*PL-NP.sqrt((2*l+1.0)/(2*l-1.0))*PLm1)/(1.0-x*x)
    result[x== 1] = l*(l+1)/2. * NP.sqrt((2*l+1.e0)/2.e0)
    result[x==-1] =  (-1)**(L+1)*l*(l+1)/2. * NP.sqrt((2*l+1.e0)/2.e0)
  else:
    result        = -l*(x*PL-PLm1)/(1-x*x)
    result[x== 1] = l*(l+1)/2.
    result[x==-1] = (-1)**(L+1)*l*(l+1)/2.
   
  return result

def iterLAssociatedLegendre(L,M,x,PLm1M,PLm2M,normalized=True):

  l = float(L)
  m = float(M)
  if normalized:
    result  = (2*l-1)*NP.sqrt((2*l+1)/(2*l-1.0)*(l-m)/(l+m*1.0))*x*PLm1M
    result -= (l+m-1)*NP.sqrt((2*l+1)/(2*l-3.0)*(l-m)*(l-m-1.0)/((l+m)*(l+m-1.0)))*PLm2M
    result /= l-m*1.0

  else:
    result = ((2*l-1)*x*PLm1M - (l+m-1)*PLm2M)/(l-m)
  return result

def iterMAssociatedLegendre(L,M,x,PLMm1,PLMm2,normalized=True):

  l = float(L)
  m = float(M)
  result = NP.zeros(PLMm1.shape)
  if normalized:
    result[x!=0] = -2*(m-1)/NP.sqrt(1-x[x!=0]**2)*x[x!=0]*PLMm1[x!=0]\
                   -(l+m-1)*(l-m+2)*PLMm2[x!=0]
  else:
    result[x!=0]  = -2*(m-1)/NP.sqrt(1-x[x!=0]**2)*x[x!=0]*PLMm1[x!=0] - PLMm2[x!=0]
    result       *= 1.e0/NP.sqrt((l+m)*(l-m+1.))
  return result

def associatedLegendre_grid(ls,ms,theta=None,normalized=True):


  # Theta
  if theta is None:
    th = NP.linspace(0,NP.pi,1000)
  else:
    th = theta

  # Set geometry and modes
  if isinstance(ls,int):
    Ls = [ls]
  else:
    Ls = ls
  if isinstance(ms,int):
    Ms = [ms]
  else:
    Ms = list(ms)
  x = NP.cos(th)


  # Recursion is done with Plm(Plm-1,Plm-2), from Pmm to Plm,
  # so the outer loop is in m. We get Pl-m at the same time

  res = []

  for im in range(len(Ms)):

    m  = Ms[im]
    am = abs(m)

    # Get Pmm and Pm+1m to start recursion (l=m)
    if am == 0:
      if normalized:
        pLM = 1/NP.sqrt(2.0)*NP.ones(x.shape)
      else:
        pLM = 1.e0*NP.ones(x.shape)
    else:
      z = NP.sqrt(1-x**2)
      if normalized:
        pLM = (-1)**am*z**am*NP.sqrt(oddOnEvenFactorial(am)/2.0)
      else:
        pLM = (-1)**am*oddFactorial(2*am-1)*z**am

    if normalized:
      pLp1M = pLM*x*NP.sqrt(2.0*am+3)
    else:
      pLp1M = pLM*x*(2*am+1)

    # Loop in L
    Lnow = am+2
    resm = []
    for il in range(len(Ls)):

      # Compute only if l>=m
      l = Ls[il]

      if l >= am:

        # Get appropriate associated Legendre polynomial
        if l==am:
          ALgdr = pLM
        elif l==am+1:
          ALgdr = pLp1M
        else:
          while(Lnow <= l):
            pLp2M = iterLAssociatedLegendre(Lnow,am,x,pLp1M,pLM,normalized)
            pLM   = pLp1M
            pLp1M = pLp2M
            Lnow += 1
            ALgdr = pLp2M

        if m<0:
          if normalized:
            ALgdr = ALgdr*(-1)**am
          else:
            ALgdr = ALgdr*(-1)**am/(1.e0*NP.product(NP.arange(l-am+1,l+am+1)))
        resm.append(ALgdr)
        del ALgdr
      else:
        resm.append(NP.zeros(th.shape)*NP.nan)
    res.append(NP.array(resm))
 
  # Revert axes to their previous place
  #res = NP.moveaxis(res,-1,axisTheta)
  return NP.swapaxes(NP.array(res),0,1)
